Дамашнее задание

In [ ]:
import pandas as pd
import numpy as np

import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
!pip install razdel

In [ ]:
!pip install pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 67.3 MB/s eta 0:00:00


In [ ]:
# Для токенизации
from razdel import tokenize

# Для лемматизации
import pymorphy3
morph = pymorphy3.MorphAnalyzer()

# Для стемминга
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")

# Модели
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

# Метрики
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

In [ ]:
df = pd.read_csv('ru_sentiment_dataset.csv')

print(df.head())

                                                text  label
0  Развода на деньги нет Наблюдаюсь в Лайфклиник ...      1
1  Отель выбрали потому что рядом со стадионом. О...      0
2  Вылечили Гноился с рождения глазик, в поликлин...      1
3  Хорошее расположение.С вокзала дошли пешком.Но...      0
4  Отличное месторасположение,прекрасный вид,особ...      1


In [ ]:
print(df['label'].value_counts())

label
1    10026
0     5560
2     5512
Name: count, dtype: int64


Препроцессинг

In [ ]:
# Задание 1. Очистить текст удалить всё, что не слова, поменять на нижний регистр
def clean_text(text):

    text = re.sub(r'[^a-zA-Zа-яА-ЯёЁ ]', ' ', text)
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [ ]:
df['text'] = df['text'].astype(str)
df['cleaned'] = df['text'].apply(clean_text)

In [ ]:
df[['text','cleaned']].head()

,text,cleaned
0,Развода на деньги нет Наблюдаюсь в Лайфклиник ...,развода на деньги нет наблюдаюсь в лайфклиник ...
1,Отель выбрали потому что рядом со стадионом. О...,отель выбрали потому что рядом со стадионом от...
2,"Вылечили Гноился с рождения глазик, в поликлин...",вылечили гноился с рождения глазик в поликлини...
3,Хорошее расположение.С вокзала дошли пешком.Но...,хорошее расположение с вокзала дошли пешком но...
4,"Отличное месторасположение,прекрасный вид,особ...",отличное месторасположение прекрасный вид особ...


In [ ]:
# Задание 2. токенизация

def tokenize_text(text):

    tokens = [t.text for t in tokenize(text)]
    return tokens

df['tokens'] = df['cleaned'].apply(tokenize_text)

In [ ]:
df[['cleaned','tokens']].head()

,cleaned,tokens
0,развода на деньги нет наблюдаюсь в лайфклиник ...,"[развода, на, деньги, нет, наблюдаюсь, в, лайф..."
1,отель выбрали потому что рядом со стадионом от...,"[отель, выбрали, потому, что, рядом, со, стади..."
2,вылечили гноился с рождения глазик в поликлини...,"[вылечили, гноился, с, рождения, глазик, в, по..."
3,хорошее расположение с вокзала дошли пешком но...,"[хорошее, расположение, с, вокзала, дошли, пеш..."
4,отличное месторасположение прекрасный вид особ...,"[отличное, месторасположение, прекрасный, вид,..."


In [ ]:
# Задание 3. лемматизация

def lemmatize_tokens(tokens):
    lemmas = []
    for token in tokens:
        p = morph.parse(token)[0]
        lemmas.append(p.normal_form)
    return lemmas

df['lemmas'] = df['tokens'].apply(lemmatize_tokens)


In [ ]:
df['lemmas_text'] = df['lemmas'].apply(lambda x: " ".join(x))

Train/Test split

In [ ]:
X = df['lemmas_text']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [ ]:
# Задание 4. векторизация — TF-IDF

tfidf = TfidfVectorizer(
    min_df=5,
    max_features=10000
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf  = tfidf.transform(X_test)

## Обучение моделей

In [ ]:
# Multinomial Naive Bayes

nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)

pred_nb = nb.predict(X_test_tfidf)

print("MultinomialNB")
print("Accuracy:", accuracy_score(y_test, pred_nb))
print("F1 macro:", f1_score(y_test, pred_nb, average='macro'))
print(confusion_matrix(y_test, pred_nb))
print(classification_report(y_test, pred_nb))


MultinomialNB
Accuracy: 0.657345971563981
F1 macro: 0.6424506869344359
[[ 637  338  137]
 [ 403 1465  137]
 [ 129  302  672]]
              precision    recall  f1-score   support

           0       0.54      0.57      0.56      1112
           1       0.70      0.73      0.71      2005
           2       0.71      0.61      0.66      1103

    accuracy                           0.66      4220
   macro avg       0.65      0.64      0.64      4220
weighted avg       0.66      0.66      0.66      4220



In [ ]:
# Logistic Regression

logreg = LogisticRegression(max_iter=300, class_weight='balanced')
logreg.fit(X_train_tfidf, y_train)

pred_lr = logreg.predict(X_test_tfidf)

print("Logistic Regression")
print()
print("Accuracy:", accuracy_score(y_test, pred_lr))
print("F1 macro:", f1_score(y_test, pred_lr, average='macro'))
print(confusion_matrix(y_test, pred_lr))
print(classification_report(y_test, pred_lr))


Logistic Regression

Accuracy: 0.7075829383886256
F1 macro: 0.7054514026068416
[[ 815  160  137]
 [ 493 1314  198]
 [ 146  100  857]]
              precision    recall  f1-score   support

           0       0.56      0.73      0.64      1112
           1       0.83      0.66      0.73      2005
           2       0.72      0.78      0.75      1103

    accuracy                           0.71      4220
   macro avg       0.70      0.72      0.71      4220
weighted avg       0.73      0.71      0.71      4220



In [ ]:
# Linear SVM

svm_clf = LinearSVC(class_weight='balanced')
svm_clf.fit(X_train_tfidf, y_train)

pred_svm = svm_clf.predict(X_test_tfidf)

print("Linear SVM")
print()
print("Accuracy:", accuracy_score(y_test, pred_svm))
print("F1 macro:", f1_score(y_test, pred_svm, average='macro'))
print(confusion_matrix(y_test, pred_svm))
print(classification_report(y_test, pred_svm))


Linear SVM

Accuracy: 0.6914691943127962
F1 macro: 0.6787050949578436
[[ 654  299  159]
 [ 367 1463  175]
 [ 138  164  801]]
              precision    recall  f1-score   support

           0       0.56      0.59      0.58      1112
           1       0.76      0.73      0.74      2005
           2       0.71      0.73      0.72      1103

    accuracy                           0.69      4220
   macro avg       0.68      0.68      0.68      4220
weighted avg       0.69      0.69      0.69      4220

